In [125]:
import copy
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()

In [293]:
class Layer(tf.Module):
    def __init__(self, s_weights, n_neurons=3, activation=tf.identity):
        self.n_neurons = n_neurons
        self.s_weights = s_weights
        self.activation = activation
        self.build = False
        self.normalized = False
    
    @tf.function(reduce_retracing=True)
    def xavier_init(self, shape):
        in_dim, out_dim = shape
        xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
        weight_vals = tf.cast(tf.random.uniform(shape=(in_dim, out_dim), 
                                        minval=-xavier_lim, maxval=xavier_lim, seed=22, dtype=tf.float32), dtype=tf.double)
        return weight_vals

    def __call__(self, X, y):
        if not self.build:
            n_samples, n_features = X.shape
            n_output = y.shape[0]
            self.weights = tf.Variable(self.xavier_init(shape=(n_features, self.s_weights)), name="Weights", dtype=tf.double, trainable=True )
            self.bias = tf.Variable(tf.zeros(shape=self.s_weights, dtype=tf.double), name="Bias", dtype=tf.double, trainable=True )
            self.build = True
            z = tf.add(tf.matmul(X, self.weights), self.bias)
            return self.activation(z)

    

class MLPLinearRegressor(tf.Module):
    def __init__(self, X, y, layers, lr=0.001):
        self.X = X
        self.y = y
        self.build = False
        self.layers = layers
        self.optimizer = tf.keras.optimizers.Adam(lr)
    
    @tf.function(reduce_retracing=True)
    def accuracy_mae(self, predicted):
        return tf.divide(tf.reduce_sum(tf.abs(tf.subtract(predicted , self.y))),len(predicted))
    
    @tf.function(reduce_retracing=True)
    def accuracy_rae(self, predicted):
        return tf.divide(
            tf.reduce_sum(tf.abs(tf.subtract(predicted, tf.sigmoid(self.y)))),
            tf.reduce_sum(tf.abs(tf.reduce_mean(tf.subtract(tf.sigmoid(self.y), predicted))))
        )

    @tf.function(reduce_retracing=True)
    def accuracy_rrsse(self, predicted):
        return tf.divide(tf.sqrt(
            tf.divide(
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(predicted, tf.sigmoid(self.y)))))),
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(tf.reduce_mean(tf.sigmoid(self.y)), predicted)))))
            )), len(predicted)
        )

    """@tf.function
    def normalize_dataset(self, X):
        return tf.keras.utils.normalize(X)"""


    @tf.function(reduce_retracing=True)
    def loss(self, predicted):
        return 1/2 * tf.reduce_sum( tf.square(tf.subtract(tf.sigmoid(self.y), tf.squeeze(predicted))))

    def train_step(self):
        for layer in self.layers:
            self.X = layer(self.X, self.y)
            print(self.X)
        return self.X
           
    def train(self, epochs=1, lr=0.001):
        for e in range(epochs):
            with tf.GradientTape() as tape:
                y_pred = model.train_step()
                loss = self.loss(y_pred)
            acc_mae = self.accuracy_mae(y_pred)
            acc_rae = self.accuracy_rae(y_pred)
            acc_rrse = self.accuracy_rrsse(y_pred)
            self.vars = [self.layers[0].weights, self.layers[1].weights, self.layers[2].weights, self.layers[0].bias, self.layers[1].bias, self.layers[2].bias]
            grads = tape.gradient(loss, self.vars) 
            self.optimizer.apply_gradients(zip(grads, self.vars))

        """print('acc_mae: ',acc_mae)
        print('acc_rae: ',acc_rae)
        print('acc_rrse: ',acc_rrse)
        print('loss: ', loss)"""

        print('predicted:', tf.squeeze(y_pred))
        print('y', tf.sigmoid(self.y))


X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.95, random_state=2)

model = MLPLinearRegressor(X_train, y_train, [
    Layer(20,activation=tf.sigmoid),
    Layer(20,activation=tf.sigmoid),
    Layer(1,)
])
model.train(epochs=2, lr=0.001)


tf.Tensor(
[[1.00000000e+000 1.00000000e+000 3.26897913e-081 ... 3.76127895e-022
  1.00000000e+000 1.41934341e-090]
 [1.00000000e+000 1.00000000e+000 1.08008426e-118 ... 3.73651242e-041
  1.00000000e+000 5.21618989e-136]
 [1.00000000e+000 1.00000000e+000 3.24781228e-101 ... 6.66802894e-033
  1.00000000e+000 5.14854085e-115]
 ...
 [1.00000000e+000 1.00000000e+000 3.59364383e-128 ... 5.13022043e-047
  1.00000000e+000 3.22222150e-147]
 [1.00000000e+000 1.00000000e+000 8.30889184e-269 ... 6.27797526e-112
  1.00000000e+000 0.00000000e+000]
 [1.00000000e+000 1.00000000e+000 1.02096725e-134 ... 1.15398956e-047
  1.00000000e+000 1.07600995e-155]], shape=(1032, 20), dtype=float64)
tf.Tensor(
[[0.53221924 0.54729535 0.6595614  ... 0.25967811 0.62627124 0.4058832 ]
 [0.53221928 0.54729538 0.65956141 ... 0.25967807 0.62627127 0.40588317]
 [0.53221928 0.54729538 0.65956141 ... 0.25967807 0.62627127 0.40588317]
 ...
 [0.53221928 0.54729538 0.65956141 ... 0.25967807 0.62627127 0.40588317]
 [0.5322192

ValueError: in user code:

    File "C:\Users\Utente\AppData\Local\Temp\ipykernel_5124\3964448437.py", line 64, in loss  *
        return 1/2 * tf.reduce_sum( tf.square(tf.subtract(tf.sigmoid(self.y), tf.squeeze(predicted))))

    ValueError: Tried to convert 'input' to a tensor and failed. Error: None values not supported.
